<a href="https://colab.research.google.com/github/pdpavandesai/MergingDiabetesData/blob/main/Trained_Model_For_Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import warnings
from numpy.lib.polynomial import poly
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
# Uploading symptomatic featured data and clinical test data

from google.colab import files
uploaded = files.upload()

Saving symptomatic_dataset.csv to symptomatic_dataset.csv
Saving clinical_test_dataset.csv to clinical_test_dataset.csv


In [3]:
# Reading symptomatic featured data and clinical test data
symptomatic_data = pd.read_csv('symptomatic_dataset.csv')
clinical_test_data = pd.read_csv('clinical_test_dataset.csv')

In [4]:
# Creating a copy of above data
sd_copy = symptomatic_data.copy()
ctd_copy = clinical_test_data.copy()

In [5]:
# Splitting symptomatic_data into training & testing data
X_train_sd, X_test_sd, y_train_sd, y_test_sd = train_test_split(sd_copy.drop('Diabetes', axis=1), sd_copy['Diabetes'], test_size=0.25, random_state=42)

In [6]:
# Splitting clinical_test_data into training & testing data
X_train_ctd, X_test_ctd, y_train_ctd, y_test_ctd = train_test_split(ctd_copy.drop('Diabetes', axis=1), ctd_copy['Diabetes'], test_size=0.25, random_state=42)

In [7]:
# Training Decision Tree classifier for symptomatic dataset
X_train_sd, X_test_sd, y_train_sd, y_test_sd = train_test_split(sd_copy.drop('Diabetes', axis=1), sd_copy['Diabetes'], test_size=0.25, random_state=42)
model_sd = DecisionTreeClassifier()
model_sd.fit(X_train_sd, y_train_sd)

DecisionTreeClassifier()

In [16]:
# Training Multi-Layer Perceptron classifier for clinical feature dataset
X_train_ctd, X_test_ctd, y_train_ctd, y_test_ctd = train_test_split(ctd_copy.drop('Diabetes', axis=1), ctd_copy['Diabetes'], test_size=0.25, random_state=42)
model_ctd = MLPClassifier(hidden_layer_sizes=(128, 64, 32), activation='logistic', solver='lbfgs', max_iter= 2000)
model_ctd.fit(X_train_ctd, y_train_ctd)

MLPClassifier(activation='logistic', hidden_layer_sizes=(128, 64, 32),
              max_iter=2000, solver='lbfgs')

In [8]:
def patientSymptomaticToDiabetes(polyuria, polydipsia, suddenWeightLoss, partialParesis, polyphagia, visualBlurring, irritability):
  patient_symptoms = pd.DataFrame({'Polyuria': [polyuria], 'Polydipsia': [polydipsia], 'SuddenWeightLoss': [suddenWeightLoss], 'PartialParesis': [partialParesis]
                             ,'Polyphagia': [polyphagia],'Irritability':[irritability],'VisualBlurring':[visualBlurring]})
  symptoms_based_prediction = model_sd.predict(patient_symptoms)
  if symptoms_based_prediction == 1:
    print('Recommended below medical tests:\n')
    print('1. Blood sugar test\n')
    print('2. Hb1Aclevel\n')
    print('3. Blood pressure test\n')
    print('4. Cholestrol test\n')
    print('5. BMI\n')
  else:
    print('Recommended to consult a doctor as the symptoms are not leading to diabetes\n')

  return symptoms_based_prediction

In [17]:
def confirmingDiabetesAndProvidingRecommendation(age, bmi, haveHighBP, haveHighCholestrol, blood_glucose_level, patient):
  patient_clinical_information = pd.DataFrame({'Age': [age], 'BMI': [bmi], 'HighChol': [haveHighBP], 'HighBP': [haveHighCholestrol]})
  clinical_diabetic_confirmation = model_ctd.predict(patient_clinical_information)
  return clinical_diabetic_confirmation

In [9]:
def categorizeAge(age):
  if age>=18 and age<=24:
    return 1
  elif age>=25 and age<=29:
    return 2
  elif age>=30 and age<=34:
    return 3
  elif age>=35 and age<=39:
    return 4
  elif age>=40 and age<=44:
    return 5
  elif age>=45 and age<=49:
    return 6
  elif age>=50 and age<=54:
    return 7
  elif age>=55 and age<=59:
    return 8
  elif age>=60 and age<=64:
    return 9
  elif age>=65 and age<=69:
    return 10
  elif age>= 70 and age<=74:
    return 11
  elif age>=75 and age<=79:
    return 12
  elif age>=80:
    return 13
  else:
    return 0

In [18]:
def is_patient_symptomatic(answer_to_symptom):
  answer = 0;
  if answer_to_symptom.lower() == 'yes':
    answer = 1
  return answer

def does_patient_has_HighBP(sbp, dbp):
  high_bp = 0
  if(sbp>=130 and dbp>=80):
    high_bp = 1
  return high_bp

def does_patient_has_HighChol(total_cholesterol_level):
  high_chol = 0
  if total_cholesterol_level >= 200:
    high_chol = 1
  return high_chol

In [31]:
def hypertension_type(sbp, dbp):
  hyp_type = ''
  if((sbp>=130 and sbp<=139) and (dbp>80 and dbp<90)):
    hyp_type = 'Stage-1 Hypertension'
    return hyp_type
  elif(sbp>=140 and (dbp>=90 and dbp<110)):
    hyp_type = 'Stage-1 Hypertension'
    return hyp_type
  else:
    hyp_type = 'Hypertensive crisis'
    return hyp_type

def diabetics_type(blood_sugar_level):
  if(blood_sugar_level < 100):
    return('do not have diabetes')
  elif(blood_sugar_level >= 100 and blood_sugar_level <=125):
    return('Prediabetic')
  else:
    return('diabetic')

def obesity_level(bmi):
  obese_level = ''
  if(bmi<18.5):
    obese_level = 'Underweight'
  elif(bmi>=18.5 and bmi<=24.9):
    obese_level = 'normal'
  elif(bmi>=25 and bmi<=29.9):
    obese_level = 'Overweight'
  elif(bmi>=30 and bmi<=40):
    obese_level = 'Obese'
  else:
    obese_level = 'Severe Obesity'
  return obese_level

In [27]:
def consultApp():
  print('Answer the following questions')
  is_symptomatic_to_polyuria = input('Do you urinate frequently?\n')
  polyuria = is_patient_symptomatic(is_symptomatic_to_polyuria)

  is_symptomatic_to_polydipsia = input('Do you feel thirsty all the time, even after drinking fluids?\n')
  polydipsia = is_patient_symptomatic(is_symptomatic_to_polydipsia)

  is_symptomatic_to_irritability = input('do you feel easily annoyed or angered? or Do you find yourself snapping at people over small things?\n')
  irritability = is_patient_symptomatic(is_symptomatic_to_irritability)

  is_symptomatic_to_suddenWeightLoss = input('Have you noticed a sudden drop in your weight?\n')
  suddenWeightLoss = is_patient_symptomatic(is_symptomatic_to_suddenWeightLoss)

  is_symptomatic_to_partialParesis = input('do you feel that your muscle movement has become weakened or impaired?\n')
  partialParesis = is_patient_symptomatic(is_symptomatic_to_partialParesis)

  is_symptomatic_to_polyphagia = input('Do you feel like you are constantly hungry, even after eating? or Do you find yourself eating more often than usual?\n')
  polyphagia = is_patient_symptomatic(is_symptomatic_to_polyphagia)

  is_symptomatic_to_visualBlurring = input('Have you experienced any changes in your vision, such as blurry or hazy sight?\n')
  visualBlurring = is_patient_symptomatic(is_symptomatic_to_visualBlurring)

  age = int(input('Please enter your Age\n'))
  age = categorizeAge(age)
  name = input('Please enter your Name\n')

  isSymptomaticToDiabetes = patientSymptomaticToDiabetes(polyuria, polydipsia, suddenWeightLoss, partialParesis, polyphagia, visualBlurring, irritability)
  if isSymptomaticToDiabetes == 1:
    bmi = float(input('Enter your BMI\n'))
    sbp = float(input('Enter your systolic blood pressure (mm Hg)?\n'))
    dbp = float(input('Enter your diastolic blood pressure (mm Hg)?\n'))
    haveHighBP = does_patient_has_HighBP(sbp, dbp)
    total_cholesterol_level = float(input('Enter your total cholesterol level (mg/dL)\n'))
    haveHighCholestrol = does_patient_has_HighChol(total_cholesterol_level)
    blood_sugar_level = float(input('Enter your blood sugar level(fasting)\n'))

    clinical_diabetic_confirmation = confirmingDiabetesAndProvidingRecommendation(age, bmi, haveHighBP, haveHighCholestrol, blood_sugar_level, name)
    if clinical_diabetic_confirmation == 1:
      print('Based on your age & clinical data, we conclude and recommend the following:\n')
      print('Based on your blood sugar level you are '+diabetics_type(blood_sugar_level)+'\n')
      print('Based on your BMI you are '+obesity_level(bmi)+'\n')
      print('Based on you BP levels you are '+hypertension_type(sbp, dbp)+'\n')
      if(blood_sugar_level >= 125 and haveHighBP == 1 and haveHighCholestrol == 1):
        if((age>=4 and age <=6) or (age>=9 and age<=11)):
          print('put on insulin and statins control of cholesterol by diet')
    else:
      print('Based on your age & clinical data, we conclude and recommend the following:\n')
      print('Based on your BMI you are '+obesity_level(bmi)+'\n')
      print('Based on your blood sugar level you are '+diabetics_type(blood_sugar_level)+'\n')
      print('Based on you BP levels you are '+hypertension_type(sbp, dbp)+'\n')
      print('You do not have diabetes, please consult a doctor for your mentioned symptoms')
  else:
    print('You do not have diabetes, please consult a doctor for your mentioned symptoms')



In [32]:
consultApp()

Answer the following questions
Do you urinate frequently?
no
Do you feel thirsty all the time, even after drinking fluids?
yes
do you feel easily annoyed or angered? or Do you find yourself snapping at people over small things?
no
Have you noticed a sudden drop in your weight?
no
do you feel that your muscle movement has become weakened or impaired?
no
Do you feel like you are constantly hungry, even after eating? or Do you find yourself eating more often than usual?
no
Have you experienced any changes in your vision, such as blurry or hazy sight?
no
Please enter your Age
72
Please enter your Name
Desai
Recommended below medical tests:

1. Blood sugar test

2. Hb1Aclevel

3. Blood pressure test

4. Cholestrol test

5. BMI

Enter your BMI
28
Enter your systolic blood pressure (mm Hg)?
135
Enter your diastolic blood pressure (mm Hg)?
85
Enter your total cholesterol level (mg/dL)
220
Enter your blood sugar level(fasting)
140
Based on your age & clinical data, we conclude and recommend the

In [ ]:
8